# 랭킹 페이지 크롤링

In [ ]:
import pandas as pd
import requests as req  
import json

from bs4 import BeautifulSoup as bfs
from tqdm.notebook import tqdm

## url_to_params()
- url를 주소와 쿼리스트링으로 나누고, 쿼리스트링을 params로 변환하여 리턴하는 함수

In [ ]:
def url_to_params(url):
  split_url = url.split('?')
  b_url = split_url[0]
  
  querystring = split_url[1].split('&')
  querystring2 = []
  params = {}
  for i in range(len(querystring)):
    querystring2.append(querystring[i].split('='))

  for i in range(len(querystring2)):
    params[querystring2[i][0]] = querystring2[i][1]

  return b_url ,params

## mu_total_page()
- 무신사 랭킹페이지 total_page를 리턴해주는 함수

In [ ]:
def mu_total_page(url): 
  url = url
  resp = req.get(url)
  soup = bfs(resp.text, 'lxml')
  total_page_tag = soup.select('span.totalPagingNum')
  total_page = int(total_page_tag[0].text)

  return total_page

## rank_page_info()
- 무신사 랭킹 페이지 크롤링 정보를 musinsa_dic에 append하는 함수

In [ ]:
from numpy import busday_offset
def rank_page_info(url):
  total_page = mu_total_page(url)
  b_url, q_params = url_to_params(url)

  for i in tqdm(range(1, total_page + 1)):
    if i == 12:
      break
    
    q_params['page'] = str(i)

    resp = req.get(b_url, params = q_params)
    soup = bfs(resp.text, 'lxml')

    code_tags = soup.select('#goodsRankList > li.li_box')
    rank_tags = soup.select('p.txt_num_rank')
    brand_name_tags = soup.select('p.item_title > a')
    goods_tags = soup.select('div.article_info > p.list_info > a')
    price_tags = soup.select('div.article_info > p.price')
    brand_url = soup.select('div.article_info > p.item_title > a')
    brand_en = soup.select('p.item_title a')

    for code, rank, brand, goods, price, url, en in zip(code_tags, rank_tags, brand_name_tags, goods_tags, price_tags, brand_url, brand_en):
      musinsa_dic['상품고유코드'].append(code['data-goods-no'])
      musinsa_dic['순위'].append(rank.text.strip().replace(' ', '').replace('\n', '').split('위')[0])
      musinsa_dic['브랜드명'].append(brand.text)
      musinsa_dic['상품명'].append(goods['title'])
      musinsa_dic['브랜드url'].append(url['href'])
      musinsa_dic['brandname'].append(en['href'].replace('https://www.musinsa.com/brands/',''))
      
      price_data = price.text.strip().replace(' ', '').split('\n')
      if len(price_data) == 2:
        musinsa_dic['할인전가격'].append(price_data[0])
        musinsa_dic['할인후가격'].append(price_data[1])
      elif len(price_data) == 1:
        musinsa_dic['할인전가격'].append(price_data[0])
        musinsa_dic['할인후가격'].append('not sale')



## brand_like()

In [ ]:
def brand_like():
  like_dic = musinsa_dic['brandname']
  for like in tqdm(like_dic):
    
    url = 'https://like.musinsa.com/like/api/v2/liketypes/brand/counts'

    params = {
        'relationIds': [like]
    }

    headers = {
      'referer' : 'https://store.musinsa.com/',
      'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36'
    }

    resp = req.post(url, json = params)

    dic = json.loads(resp.text)
    musinsa_dic['like'].append(dic['data']['contents']['items'][0]['count'])

# 상품 정보 크롤링

## prod_like_cate()
- 상품 좋아요 수와 카테고리, 서브카테고리, 조회수, 누적한매량을 musinsa_dic에 append하는 함수


In [ ]:
def prod_like_cate():
  prod_codes = musinsa_dic['상품고유코드']
  for code in tqdm(prod_codes):
    value = [code]

    url1 = 'https://like.musinsa.com/like/api/v1/liketypes/goods/counts'
    params1 = {
        'relationIds' : value
    }
    headers1 = {
      'referer' : 'https://store.musinsa.com/',
      'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36'
    }
    json_resp = req.p
    ost(url1, json=params1, headers=headers1)
    
    count_dic = json.loads(json_resp.text)
    musinsa_dic['Countlike'].append(count_dic['data']['contents'][0]['count'])


    url2 = 'https://store.musinsa.com/app/goods/'+ code +'?loc=goods_rank'
    headers2 = {
        'referer': 'https://www.musinsa.com/',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36'
    }
    resp = req.get(url2, headers=headers2)
    soup = bfs(resp.text, 'lxml')
    musinsa_dic['Category'].append(soup.select('meta')[7]['content'].split(':')[1][:-4].split('>')[0].strip())
    musinsa_dic['sub_Category'].append(soup.select('meta')[7]['content'].split(':')[1][:-4].split('>')[1].strip())



    url3 = 'https://store.musinsa.com/app/product/goodsview_stats/'+code+'/0?&menu=view'

    header3 = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
        'referer': 'https://store.musinsa.com/app/goods/2043036'

    }

    resp3 = requests.get(url3,headers=header3)

    dic3 = json.loads(resp3.text)

    # 조회수
    pageview = dic3['pageview']['total_categories']
    # 누적판매
    purchase = dic3['purchase']['total_categories'] 

# 실행


In [ ]:
musinsa_dic = {
    '상품고유코드' : [],
    '순위' : [],
    '브랜드명' : [],
    '상품명' : [],
    'Countlike' : [],
    '브랜드url' : [],
    '할인전가격' : [],
    '할인후가격' : [],
    'brandname' : [],
    'like': [],
    'Category': [],
    'sub_Category' : []
    
}

In [ ]:
pants_page_url, pants_params = url_to_params('https://www.musinsa.com/category/003004?d_cat_cd=003004&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page=1&display_cnt=90&sale_goods=&group_sale=&kids=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=')

In [ ]:
pants_page_url, pants_params

In [ ]:
resp = req.get('https://www.musinsa.com/category/003004?d_cat_cd=003004&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page=1&display_cnt=90&sale_goods=&group_sale=&kids=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=')

In [ ]:
resp

<Response [200]>

In [ ]:
pred_num = {
    'num' : []
}

In [ ]:
soup = bfs(resp.text, 'lxml')
tags = soup.select('div.member_price')

for tag in tags:
  pred_num['num'].append(tag['id'].replace('sPrice_', ''))

In [ ]:
url_to_params('https://www.musinsa.com/brands/musinsastandard?category3DepthCodes=&category2DepthCodes=&category1DepthCode=003&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods=&timeSale=&includeKeywords=&sortCode=3m&tags=&page=1&size=90&listViewType=small&campaignCode=&groupSale=&outletGoods=false&boutiqueGoods=')

In [ ]:
url_to_params('https://www.musinsa.com/brands/musinsastandard?category3DepthCodes=&category2DepthCodes=003004&category1DepthCode=003&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods=&timeSale=&includeKeywords=&sortCode=3m&tags=&page=1&size=90&listViewType=small&campaignCode=&groupSale=&outletGoods=false&boutiqueGoods=')